In [ ]:
############################################################################
############################################################################
# 日経225 ファストストキャスティクス

  #ライブラリーの設定
import talib as ta 
import pandas as pd 
import numpy as np 


def initialize(ctx):
    # 設定
    ctx.logger.debug("initialize() called")
    ctx.configure(
      target="jp.stock.daily",
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols": [
            "jp.stock.9983", #ファーストリテイリング
            "jp.stock.9984", #ソフトバンク
            "jp.stock.6954", #ファナック
            "jp.stock.9433", #KDDI
            "jp.stock.8028", #ファミリーマート
            "jp.stock.8035", #東京エレクトロン
            "jp.stock.4543", #テルモ
            "jp.stock.6367", #ダイキン
            "jp.stock.6971", #京セラ
            "jp.stock.9735", #セコム
          ],
          "columns": [
            #"open_price_adj",    # 始値(株式分割調整後)
            "high_price_adj",    # 高値(株式分割調整後)
            "low_price_adj",     # 安値(株式分割調整後)
            #"close_price",        # 終値
            "close_price_adj",    # 終値(株式分割調整後) 
            #"volume_adj",         # 出来高
            #"txn_volume",         # 売買代金
          ]
        }
      }
    )



    def _STOCHASTIC_FAST(data):
      df_high = data["high_price_adj"].fillna(method='ffill')
      df_low = data["low_price_adj"].fillna(method='ffill')
      df_close = data["close_price_adj"].fillna(method='ffill')

      d_fastK = dict()
      d_fastD = dict()

      for symbol in data.minor_axis:
        fastK, fastD = ta.STOCHF(df_high[symbol].values.astype(np.double), 
                                 df_low[symbol].values.astype(np.double), 
                                 df_close[symbol].values.astype(np.double), 
                                 fastk_period=20, 
                                 fastd_period=3,)
        d_fastK[symbol] = fastK
        d_fastD[symbol] = fastD

      df_fastK = pd.DataFrame(d_fastK, index=data.major_axis)
      df_fastD = pd.DataFrame(d_fastD, index=data.major_axis)

      buy_sig = (df_fastK > df_fastD) & (df_fastK.shift(1) < df_fastD.shift(1)) 
      sell_sig = (df_fastK < df_fastD) & (df_fastK.shift(1) > df_fastD.shift(1)) 

      return {
        "fastK":df_fastK, 
        "fastD": df_fastD, 
        "buy:sig":buy_sig,
        "sell:sig":sell_sig,
        }

    # シグナル登録 
    ctx.regist_signal("STOCHASTIC_FAST", _STOCHASTIC_FAST)


def handle_signals(ctx, date, current):

    done_syms = set([])

    ratio = current["buy:sig"]
    df = ratio[ratio == 1.0]
    for (sym,val) in df.items():
        if sym in done_syms:
          continue

        sec = ctx.getSecurity(sym)
        sec.order_target_percent(0.15, comment="シグナル買(%f)" % val)
        pass

    ratio = current["sell:sig"]
    df = ratio[ratio == 1.0]
    for (sym,val) in df.items():
        if sym in done_syms:
          continue
          
        sec = ctx.getSecurity(sym)
        sec.order_target_percent(0, comment="シグナル売(%f)" % val)
        pass

    pass